In [1]:
import pandas as pd
from fuzzywuzzy import fuzz

/home/armando/ambientes_python/ambiente_ai/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## MATCH CIE CATALOG WITH PATHOLOGY CATALOG

In [6]:
pathologies = pd.read_csv('/home/armando/git/insurance_case/datalake/raw/pathologies.csv')
pathologies = pathologies[['code', 'name']]
pathologies.columns = ['pathology_code', 'pathology_name']
print(pathologies.shape)
display(pathologies.head(3))

(12423, 2)


,pathology_code,pathology_name
0,A000,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO CH..."
1,A001,"COLERA DEBIDO A VIBRIO CHOLERAE O1, BIOTIPO EL..."
2,A009,COLERA NO ESPECIFICADO


In [9]:
cie_10 = pd.read_csv('/home/armando/git/insurance_case/datalake/raw/claims_dummy.csv', usecols=['CIE10', 'DIAGNOSIS'])

cie_10 = cie_10[['CIE10', 'DIAGNOSIS']].drop_duplicates()
print(cie_10.shape)
print(cie_10['CIE10'].nunique())
print(cie_10['DIAGNOSIS'].nunique())

cie_10.columns = ['cie_code', 'cie_diagnosis']
display(cie_10.head(3))

(46, 2)
46
46


,cie_code,cie_diagnosis
0,H33,DESPRENDIMIENTO Y DESGARRO DE LA RETINA
1,D04.8,CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS E...
2,L03.9,CELULITIS DE SITIO NO ESPECIFICADO


### Match

In [14]:
array_pathology_code = []
array_cie_code = []
array_similarity = []

for index, row in cie_10.iterrows():

    diagnostic = row['cie_diagnosis']
    best_match = None
    highest_ratio = 0
    pathology_code = None
    array_cie_code.append(row['cie_code'])

    for _, patho in pathologies.iterrows():
        patho_name = patho['pathology_name']
        ratio = fuzz.ratio(diagnostic, patho_name)

        if ratio > highest_ratio:
            highest_ratio = ratio
            best_match = patho_name
            path_code = patho['pathology_code']

    array_pathology_code.append(path_code)
    array_similarity.append(highest_ratio)
    print(f"Diagnosis: {diagnostic}, Best Match: {best_match}, Similarity: {highest_ratio}")

Diagnosis: DESPRENDIMIENTO Y DESGARRO DE LA RETINA, Best Match: DESPRENDIMIENTO SEROSO DE LA RETINA, Similarity: 86
Diagnosis: CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS ESPECIFICADOS, Best Match: CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS ESPECIFICADOS, Similarity: 100
Diagnosis: CELULITIS DE SITIO NO ESPECIFICADO, Best Match: CELULITIS DE SITIO NO ESPECIFICADO, Similarity: 100
Diagnosis: CALCULO DEL URETER, Best Match: CALCULO DEL URETER, Similarity: 100
Diagnosis: CALCULO DEL RI¿ON, Best Match: CALCULO DEL RIÑON, Similarity: 94
Diagnosis: OTROS TRASTORNOS DE LA NARIZ Y DE LOS SENOS PARANASALES, Best Match: OTROS TRASTORNOS ESPECIFICADOS DE LA NARIZ Y DE LOS SENOS PARANASALES, Similarity: 89
Diagnosis: HERNIA INGUINAL UNILATERAL O NO ESPECIFICADA SIN OBSTRUCCION NI GANGRENA, Best Match: HERNIA INGUINAL UNILATERAL O NO ESPECIFICADA, SIN OBSTRUCION NI GANGRENA, Similarity: 99
Diagnosis: CARDIOMIOPATIA ISQUEMICA, Best Match: CARDIOMIOPATIA ISQUEMICA, Similarity: 100
Diagnosis: OTRAS

In [19]:
#ENSEMBLE RESULT
cat_cie_path = pd.DataFrame({'cie_code': array_cie_code, 
                             'pathology_code': array_pathology_code,
                             'similarity': array_similarity})

cat_cie_path = cat_cie_path.merge(pathologies, on='pathology_code', how='inner')
cat_cie_path = cat_cie_path.merge(cie_10, on='cie_code', how='inner')

print(cat_cie_path.shape)
display(cat_cie_path.head(3))

for index, row in cat_cie_path.iterrows():
    print(index, 'Similarity:', row['similarity'])
    print(row['cie_diagnosis'])
    print(row['pathology_name'])
    print('---')

(46, 5)


,cie_code,pathology_code,similarity,pathology_name,cie_diagnosis
0,H33,H332,86,DESPRENDIMIENTO SEROSO DE LA RETINA,DESPRENDIMIENTO Y DESGARRO DE LA RETINA
1,D04.8,D048,100,CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS E...,CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS E...
2,L03.9,L039,100,CELULITIS DE SITIO NO ESPECIFICADO,CELULITIS DE SITIO NO ESPECIFICADO


0 Similarity: 86
DESPRENDIMIENTO Y DESGARRO DE LA RETINA
DESPRENDIMIENTO SEROSO DE LA RETINA
---
1 Similarity: 100
CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS ESPECIFICADOS
CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS ESPECIFICADOS
---
2 Similarity: 100
CELULITIS DE SITIO NO ESPECIFICADO
CELULITIS DE SITIO NO ESPECIFICADO
---
3 Similarity: 100
CALCULO DEL URETER
CALCULO DEL URETER
---
4 Similarity: 94
CALCULO DEL RI¿ON
CALCULO DEL RIÑON
---
5 Similarity: 89
OTROS TRASTORNOS DE LA NARIZ Y DE LOS SENOS PARANASALES
OTROS TRASTORNOS ESPECIFICADOS DE LA NARIZ Y DE LOS SENOS PARANASALES
---
6 Similarity: 99
HERNIA INGUINAL UNILATERAL O NO ESPECIFICADA SIN OBSTRUCCION NI GANGRENA
HERNIA INGUINAL UNILATERAL O NO ESPECIFICADA, SIN OBSTRUCION NI GANGRENA
---
7 Similarity: 100
CARDIOMIOPATIA ISQUEMICA
CARDIOMIOPATIA ISQUEMICA
---
8 Similarity: 83
OTRAS ANEMIAS
OTRAS TALASEMIAS
---
9 Similarity: 63
COVID-19, VIRUS IDENTIFICADO
INFLUENZA CON NEUMONIA, VIRUS NO IDENTIFICADO
---
10 Similarity: 100
N

In [20]:
discard = ['DESPRENDIMIENTO Y DESGARRO DE LA RETINA', 'OTRAS ANEMIAS',
           'COVID-19, VIRUS IDENTIFICADO', 'INFARTO AGUDO DEL MIOCARDIO',
           'CHOQUE SÉPTICO', 'HERNIA FEMORAL', 'GASTROENTERITIS Y COLITIS DE ORIGEN NO ESPECIFICADO',
           'APENDICITIS AGUDA']

cat_cie_path_final = cat_cie_path[~cat_cie_path['cie_diagnosis'].isin(discard)]
print(cat_cie_path_final.shape)
display(cat_cie_path_final.head(3))

(38, 5)


,cie_code,pathology_code,similarity,pathology_name,cie_diagnosis
1,D04.8,D048,100,CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS E...,CARCINOMA IN SITU DE LA PIEL DE OTROS SITIOS E...
2,L03.9,L039,100,CELULITIS DE SITIO NO ESPECIFICADO,CELULITIS DE SITIO NO ESPECIFICADO
3,N20.1,N201,100,CALCULO DEL URETER,CALCULO DEL URETER


In [21]:
cat_cie_path_final.to_csv('/home/armando/git/insurance_case/datalake/preprocessed/cie10_to_pathology_mapping.csv', index=False)